In [72]:
from qibo.models.variational import VQE
# from boostvqe.ansatze import build_circuit
from qibo import hamiltonians, Circuit,gates
from pathlib import Path
import numpy as np
import scipy

## CNOT + single qubit rotation decomposition of each 2-qubit XXZ evolution for n-qubit XXZ Trotterisation

In [73]:
def single_XXZ_decomposition(nqubits,q_i,q_ip1,t,delta):
#     This is a code that decomposes e^{-it(XX + YY + delta*ZZ)}
    alpha = -t
    beta = -t
    gamma = -delta*t
    circuit = Circuit(nqubits = nqubits)
    circuit.add(gates.RZ(q_ip1, -np.pi/2))
    circuit.add(gates.CNOT(q_ip1, q_i))
    circuit.add(gates.RZ(q_i, -2*gamma + np.pi/2))
    circuit.add(gates.RY(q_ip1, -np.pi/2+2*alpha))
    circuit.add(gates.CNOT(q_i, q_ip1))
    circuit.add(gates.RY(q_ip1, -2*beta + np.pi/2))
    circuit.add(gates.CNOT(q_ip1, q_i))
    circuit.add(gates.RZ(q_i, np.pi/2))
    return circuit

## Test for 2-qubit XXZ

In [74]:
delta = 3
t = 1
# Multiplying 0.5 to h_xxz for making it h_XXZ = (XX + YY + delta*ZZ)
# Without this multiplication, we get h_XXZ = 2(XX + YY + delta*ZZ) due to the periodic BC of hamiltonians.XXZ
h_xxz = 0.5*hamiltonians.XXZ(nqubits=2, delta = delta)
circ = single_XXZ_decomposition(nqubits = 2,q_i = 0,q_ip1 = 1,t = t,delta = delta)

In [75]:
u = h_xxz.exp(t)
v = circ.unitary()
print(v)
print(u)
np.linalg.norm(u-(1+1j)*v/np.sqrt(2))

[[-7.99817322e-01+6.00243493e-01j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j -1.96261557e-17-1.96261557e-17j]
 [ 0.00000000e+00+0.00000000e+00j  2.49789431e-01-3.32841449e-01j
   7.27271833e-01+5.45799864e-01j  0.00000000e+00+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j  7.27271833e-01+5.45799864e-01j
   2.49789431e-01-3.32841449e-01j  0.00000000e+00+0.00000000e+00j]
 [-1.96261557e-17-1.96261557e-17j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j -7.99817322e-01+6.00243493e-01j]]
[[-0.9899925 -0.14112001j  0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.41198225-0.05872664j  0.12832006+0.90019763j
   0.        +0.j        ]
 [ 0.        +0.j          0.12832006+0.90019763j  0.41198225-0.05872664j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
  -0.9899925 -0.14112001j]]


7.838800601857069e-16

## n-qubit function (even qubits for now)

In [111]:
def nqubit_XXZ_decomposition(nqubits,t,delta,steps):
    dt = t/steps
    alpha = -dt
    beta = -dt
    gamma = -delta*dt
    circuit = Circuit(nqubits = nqubits)
    even_qubits = np.arange(0,nqubits,2)
    odd_qubits = np.arange(1,nqubits,2)
    if nqubits%2 == 0:
#         commented out RZ rotations that cancel out with each other
#         circuit.add(gates.RZ(q_i+1, -np.pi/2) for q_i in even_qubits)
        circuit.add(gates.CNOT(q_i+1, q_i) for q_i in even_qubits)
        circuit.add(gates.RZ(q_i, -2*gamma + np.pi/2) for q_i in even_qubits)
        circuit.add(gates.RY(q_i+1, -np.pi/2+2*alpha) for q_i in even_qubits)
        circuit.add(gates.CNOT(q_i, q_i+1) for q_i in even_qubits)
        circuit.add(gates.RY(q_i+1, -2*beta + np.pi/2) for q_i in even_qubits)
        circuit.add(gates.CNOT(q_i+1, q_i) for q_i in even_qubits)
        
        circuit.add(gates.CNOT(even_qubits[n],odd_qubits[n-1]) for n in range(nqubits//2))
        circuit.add(gates.RZ(q_i+1, -2*gamma + np.pi/2) for q_i in even_qubits)
        circuit.add(gates.RY(q_i, -np.pi/2+2*alpha) for q_i in even_qubits)
        circuit.add(gates.CNOT(odd_qubits[n-1],even_qubits[n]) for n in range(nqubits//2))
        circuit.add(gates.RY(q_i, -2*beta + np.pi/2) for q_i in even_qubits)
        circuit.add(gates.CNOT(even_qubits[n],odd_qubits[n-1]) for n in range(nqubits//2))
#         circuit.add(gates.RZ(q_i+1, np.pi/2) for q_i in even_qubits)
        
        multi_layer = Circuit(nqubits = nqubits)
        multi_layer.add(gates.RZ(q_i+1, -np.pi/2) for q_i in even_qubits)
        for step in range(steps):
            multi_layer += circuit
        multi_layer.add(gates.RZ(q_i+1, np.pi/2) for q_i in even_qubits)
        return multi_layer

q0: ────X─RZ─o────X─RZ─
q1: ─RZ─o─RY─X─RY─o────


In [132]:
t = 1
steps = 20
delta=0.5
nqubits=4

h_xxz = hamiltonians.XXZ(nqubits=nqubits, delta = delta)
u = h_xxz.exp(t)
circ = nqubit_XXZ_decomposition(nqubits=nqubits,t=t,delta=delta,steps=steps)
v = circ.unitary()
np.linalg.norm(u-np.exp(nqubits*steps*1j*np.pi/4)*v)

0.06976539623568771